# Inspecting Model Equations

One of the key features of ANDES is its symbolic-numeric hybrid framework, which allows you to inspect the mathematical equations that define each model. This capability is invaluable for understanding model behavior, debugging simulations, and verifying that models are implemented correctly.

Every model in ANDES is defined symbolically using SymPy, and the framework automatically generates numerical code for efficient simulation. This tutorial shows how to access and examine the symbolic equations, variables, and services that comprise each model.

In [ ]:
# Reduce logging verbosity for PDF builds
import os
if os.environ.get('SPHINX_BUILD_PDF'):
    import andes
    _orig_config_logger = andes.config_logger
    def _quiet_logger(stream_level=20, *args, **kwargs):
        stream_level = max(stream_level, 30)
        return _orig_config_logger(stream_level, *args, **kwargs)
    andes.config_logger = _quiet_logger

## Setup

In [1]:
%matplotlib inline

import andes

andes.config_logger(30)  # Reduce logging verbosity

## Listing Available Models

To see all models supported by ANDES, create an empty System object and call `supported_models()`. Models are organized into groups based on their function in the power system.

In [2]:
ss = andes.System()
print(ss.supported_models())

Supported Groups and Models

      Group        |                          Models                          
-------------------+----------------------------------------------------------
 ACLine            | Line                                                     
 ACShort           | Jumper                                                   
 ACTopology        | Bus                                                      
 Calculation       | ACE, ACEc, COI                                           
 Collection        | Area                                                     
 DCLink            | Ground, R, L, C, RCp, RCs, RLs, RLCs, RLCp               
 DCTopology        | Node                                                     
 DG                | PVD1, ESD1, EV1, EV2                                     
 DGProtection      | DGPRCT1, DGPRCTExt                                       
 DataSeries        | TimeSeries                                               
 DynLoad           | ZI

## Preparing Model Equations

Before inspecting a model's symbolic equations, you need to call `prepare()` on that model. This triggers the symbolic processing that generates the equations and Jacobian matrices. For a single model, this is fast; preparing all models takes longer but is still manageable.

In [3]:
# Prepare equations for the GENCLS model (classical generator)
ss.GENCLS.prepare()

## Model Documentation

Each model provides comprehensive documentation through the `doc()` method. This returns a formatted string containing:
- Model description and group membership
- Parameter definitions with units and defaults
- Variable definitions (states and algebraics)
- Initialization equations
- Differential and algebraic equations
- Service definitions
- Configuration options

In [4]:
print(ss.GENCLS.doc())

Model <GENCLS> in Group <SynGen>
Classical generator model.

Parameters

 Name   |       Description        | Default | Unit |        Properties       
--------+--------------------------+---------+------+-------------------------
 idx    | unique device idx        |         |      |                         
 u      | connection status        | 1       | bool |                         
 name   | device name              |         |      |                         
 bus    | interface bus id         |         |      | mandatory               
 gen    | static generator index   |         |      | mandatory               
 coi    | center of inertia index  |         |      |                         
 coi2   | center of inertia index  |         |      |                         
 Sn     | Power rating             | 100     | MVA  |                         
 Vn     | AC voltage rating        | 110     |      |                         
 fn     | rated frequency          | 60      |      |     

The documentation shows that GENCLS (classical generator) has two state variables (`delta` for rotor angle and `omega` for rotor speed) and several algebraic variables for currents, voltages, and power outputs. The differential equations implement the swing equation that governs generator dynamics.

## Accessing Variables

Model variables are stored in ordered dictionaries accessible through `states` and `algebs` attributes. This lets you enumerate all variables programmatically.

In [5]:
# State (differential) variables
print("State variables:")
for name, var in ss.GENCLS.states.items():
    print(f"  {name}: {var}")

State variables:
  delta: State: GENCLS.delta, []
  omega: State: GENCLS.omega, []


In [6]:
# Algebraic variables
print("Algebraic variables:")
for name, var in ss.GENCLS.algebs.items():
    print(f"  {name}: {var}")

Algebraic variables:
  Id: Algeb: GENCLS.Id, []
  Iq: Algeb: GENCLS.Iq, []
  vd: Algeb: GENCLS.vd, []
  vq: Algeb: GENCLS.vq, []
  tm: Algeb: GENCLS.tm, []
  te: Algeb: GENCLS.te, []
  vf: Algeb: GENCLS.vf, []
  XadIfd: Algeb: GENCLS.XadIfd, []
  Pe: Algeb: GENCLS.Pe, []
  Qe: Algeb: GENCLS.Qe, []
  psid: Algeb: GENCLS.psid, []
  psiq: Algeb: GENCLS.psiq, []


## Symbolic Equations

The symbolic equations are stored in `Model.syms`. This object contains SymPy expressions that can be displayed, manipulated, and analyzed. The main attributes are:

| Attribute | Contents |
|-----------|----------|
| `xy` | Vector of all variables (states then algebraics) |
| `f` | Differential equations (right-hand side of T x' = f) |
| `g` | Algebraic equations (right-hand side of 0 = g) |
| `df` | Jacobian df/dxy |
| `dg` | Jacobian dg/dxy |
| `s` | Service equations |

In [7]:
# All variables in order
ss.GENCLS.syms.xy

Matrix([
[ delta],
[ omega],
[    Id],
[    Iq],
[    vd],
[    vq],
[    tm],
[    te],
[    vf],
[XadIfd],
[    Pe],
[    Qe],
[  psid],
[  psiq],
[     a],
[     v]])

### Differential Equations

The differential equations define how state variables evolve over time. For generators, these typically include the swing equation relating rotor angle and speed to mechanical and electrical torque.

In [8]:
ss.GENCLS.syms.f

Matrix([
[       2*pi*fn*u*(omega - 1)],
[u*(-D*(omega - 1) - te + tm)]])

The first equation is the rotor angle dynamics: `d(delta)/dt = 2*pi*fn*(omega - 1)`, where `omega` is in per-unit of nominal frequency. The second equation is the swing equation: `M*d(omega)/dt = tm - te - D*(omega - 1)`, relating acceleration to the imbalance between mechanical torque `tm` and electrical torque `te`, with damping `D`.

### Algebraic Equations

Algebraic equations define relationships that must hold at every instant (they have no time derivative). These typically include current-voltage relationships, power calculations, and interface equations.

In [9]:
ss.GENCLS.syms.g

Matrix([
[           Id*xq + psid - vf],
[                Iq*xq + psiq],
[    -u*v*sin(a - delta) - vd],
[     u*v*cos(a - delta) - vq],
[                   -tm + tm0],
[-te + u*(-Id*psiq + Iq*psid)],
[                  u*vf0 - vf],
[             -XadIfd + u*vf0],
[     -Pe + u*(Id*vd + Iq*vq)],
[     -Qe + u*(Id*vq - Iq*vd)],
[      -psid + u*(Iq*ra + vq)],
[       psiq + u*(Id*ra + vd)],
[          -u*(Id*vd + Iq*vq)],
[          -u*(Id*vq - Iq*vd)]])

### Jacobian Matrices

The Jacobian matrices contain partial derivatives of the equations with respect to variables. These are essential for Newton-Raphson iteration in power flow and implicit integration in time-domain simulation.

In [10]:
# Jacobian of differential equations
ss.GENCLS.syms.df

Matrix([
[0, 2*pi*fn*u, 0, 0, 0, 0, 0,  0, 0, 0, 0, 0, 0, 0, 0, 0],
[0,      -D*u, 0, 0, 0, 0, u, -u, 0, 0, 0, 0, 0, 0, 0, 0]])

In [11]:
# Jacobian of algebraic equations (first 4 rows, first 6 columns)
ss.GENCLS.syms.dg[:4, :6]

Matrix([
[                 0, 0, xq,  0,  0,  0],
[                 0, 0,  0, xq,  0,  0],
[u*v*cos(a - delta), 0,  0,  0, -1,  0],
[u*v*sin(a - delta), 0,  0,  0,  0, -1]])

## Services

Services are intermediate calculations that are computed once (at initialization) and reused throughout the simulation. They often represent derived quantities like initial values or converted parameters.

In [12]:
# List all services
print("Services:")
for name, svc in ss.GENCLS.services.items():
    print(f"  {name}")

Services:
  p0
  q0
  _V
  _S
  _I
  _E
  _deltac
  delta0
  vdq
  Idq
  Id0
  Iq0
  vd0
  vq0
  tm0
  psid0
  psiq0
  vf0


In [13]:
# Service equations
ss.GENCLS.syms.s

[gammap*p0s,
 gammaq*q0s,
 v*exp(I*a),
 p0 - I*q0,
 _S/conj(_V),
 _I*(ra + I*xq) + _V,
 log(_E/Abs(_E)),
 u*im(_deltac),
 _V*u*exp(-_deltac + 0.5*I*pi),
 _I*u*exp(-_deltac + 0.5*I*pi),
 re(Idq),
 im(Idq),
 re(vdq),
 im(vdq),
 u*(Id0*(Id0*ra + vd0) + Iq0*(Iq0*ra + vq0)),
 Iq0*ra*u + vq0,
 -Id0*ra*u - vd0,
 Id0*xq + Iq0*ra + vq0]

## Practical Applications

Understanding model equations is useful for:

- **Verification**: Confirming that a model implements the expected physics
- **Debugging**: Identifying why a simulation produces unexpected results
- **Extension**: Understanding the structure before adding new features
- **Documentation**: Generating mathematical descriptions for papers or reports
- **Teaching**: Demonstrating power system dynamics concepts

## See Also

- {doc}`../modeling/concepts/framework-overview` - Overview of the symbolic-numeric framework
- {doc}`../modeling/components/variables` - Detailed variable type documentation
- {doc}`../modeling/creating-models/index` - Creating new models